# Mapped modules

`xclim` gives users the ability to generate their own modules from existing indices library. These mappings can help in emulating existing libraries (such as ICCLIM), with the added benefit of CF-compliant metadata, multilingual metadata support, and optimized calculations using federated resources (using Dask). This can be used for example to tailor existing indices with predefined thresholds without having to rewrite indices.

Presently, xclim is capable of approximating the indices developed in ICCLIM (https://icclim.readthedocs.io/en/latest/intro.html), ANUCLIM (https://fennerschool.anu.edu.au/files/anuclim61.pdf) and clix-meta (https://github.com/clix-meta/clix-meta) and is open to contributions of new indices and library mappings.

This notebook serves as an example of how one might go about creating their own library of mapped indices. Two ways are possible:

1. From a YAML file (recommended way)
2. From a mapping of indicators

## YAML file

The first method is based on the YAML syntax proposed by `clix-meta`, expanded to xclim's needs. The full documentation on that syntax is [here](../api.rst#module-xclim.core.indicator). This notebook shows a minimal example, creating a YAML file and then a module from it.

In [ ]:
!cat example.yml

`example.yml` created a module of 4 indicators.

- `RX1day` is simply the same as  `registry['RX1DAY']`, but with an updated `long_name`.
- `RX5day` is based on `registry['MAX_N_DAY_PRECIPITATION_AMOUNT']`, changed the `long_name` and injects the `window` argument.
- `R75p` is based on `registry['DAYS_OVER_PRECIP_THRESH']`, injects the `thresh` argument and changes the description of the `per` argument. Passing "data: {per}" tells xclim the value is still to be determined by the user, but other parameter's metadata field might be changed.
- `fd` is a more complex example. As there were no `base:` entry, the `Daily` class serves as a base. As it is pretty much empty, a lot has to be given explicitly:
    * A list of allowed resampling frequency is passed
    * Many output metadata fields are given
    * A index_function name if given (here it refers to a function in `xclim.indices.generic`).
    * Some parameters are injected.
    * The input variable `data` is mapped to a known variable. Function in `xclim.indices.generic` are indeed generic. Here we tell xclim that the `data` argument is minimal daily temperature. This will set the proper units check, default value and CF-compliance checks.
    
Additionnaly, the yaml specified a `realm` and `references` to be used on all indices and provided a submodule docstring. Creating the module is then simply:

In [ ]:
import xclim as xc

example = xc.core.indicator.build_indicator_module_from_yaml('example.yml', mode='warn')

In [ ]:
print(example.__doc__)
print('--')
print(xc.indicators.example.fd.__doc__)

## Mapping of indicators

For more complex mappings or if compute functions not defined in xclim are required, submodule can be constructed from Indicators directly. To construct indicator, please refer to the [customization](customize.ipynb) notebook.

In [ ]:
from xclim.core.indicator import build_indicator_module, registry
from xclim.core.utils import wrapped_partial

mapping = dict(
    egg_cooking_season=registry["MAXIMUM_CONSECUTIVE_WARM_DAYS"](
        module='awesome',
        compute=wrapped_partial(xc.indices.maximum_consecutive_tx_days, thresh="35 degC"),
        long_name="Season for outdoor egg cooking.",
    ),
    fish_feeling_days=registry["WETDAYS"](
        module='awesome',
        compute=wrapped_partial(xc.indices.wetdays, thresh="14.0 mm/day"),
        long_name="Days where we feel we are fishes"
    ),
    sweater_weather=xc.atmos.tg_min
)

awesome = build_indicator_module(
    name="awesome",
    objs=mapping,
    doc="""
        =========================
        My Awesome Custom indices
        =========================
        There are only 3 indices that really matter when you come down to brass tacks.
        This mapping library exposes them to users who want to perform real deal 
        climate science.
        """,
)

In [ ]:
print(xc.indicators.awesome.__doc__)

In [ ]:
# Let's look at our new awesome module
import inspect

print(awesome.__doc__)
for i in inspect.getmembers(awesome):
    if i[0][0] != '_':
        print(f"{i[0]} : {i[1]}")